In [ ]:
from cancer_subject.models import Appointment
from django.core.exceptions import MultipleObjectsReturned
visit_codes = {
    '1000':0,
    '1000.1':0,
    '1000.2':0,
    '1300':1,
    '1600':2,
    '1900':3,
    '1900.1':3,
    '1900.2':3,
    '2200':4,
    '2500':5,
    '2800':6,
    '3100':7,
    '3400':8,
    '3700':9,
    '4000':10,
    '4300':11,
    '4600':12,
    '4900':13,
    '5200':14,
    '5500':15,
    '5800':16,
    '6100':17,
    '6400':18,
    '6700':19,
    '7000':20
    }
visit_code_id = []
for ap in Appointment.objects.all():
    try:
        a = Appointment.objects.get(visit_code=ap.visit_code, subject_identifier=ap.subject_identifier)
    except MultipleObjectsReturned:
        visit_code_id.append([ap.visit_code, ap.subject_identifier])
for visit_code, subject_identifier in visit_code_id:
    apps = Appointment.objects.filter(visit_code=visit_code, subject_identifier=subject_identifier)
    if apps.count() == 2:
        a1, a2 = apps
        if a1.created < a2.created:
            a2.visit_code = visit_code + '.1'
            a2.save_base(raw=True)
        elif a2.created < a1.created:
            a1.visit_code = visit_code + '.1'
            a1.save_base(raw=True)
for app in Appointment.objects.all():
    app.timepoint = visit_codes.get(app.visit_code)
    app.facility_name='5-day clinic'
    app.save_base(raw=True)

In [ ]:
from edc_registration.exceptions import RegisteredSubjectError
from edc_consent.field_mixins.identity_fields_mixin import IdentityFieldsMixinError

In [17]: for line in lines:
    ...:     line = line.strip()
    ...:     line = line.split(',')
    ...:     _, subject_identifier, dob = line
    ...:     try:
    ...:         consent = SubjectConsent.objects.get(subject_identifier=subject_identifier)
    ...:     except SubjectConsent.DoesNotExist:
    ...:         print('missing consent', subject_identifier)
    ...:     else:
    ...:         dob = parser.parse(dob).replace(tzinfo=tz).date()
    ...:         consent.dob = dob
    ...:         try:
    ...:             consent.save()
    ...:         except IdentityFieldsMixinError:
    ...:             identity_problem.append(subject_identifier)
    ...:         except RegisteredSubjectError:
    ...:             identity_problem.append(subject_identifier)